In [3]:
from ppq import *
from ppq.api import *
import os
import pandas as pd

In [ ]:
fp32_acc = None
# 评估FP32准确度
if cfg.GET_FP32_ACC:
    fp32_acc,_ = get_fp32_accuracy(model_name)

for platform,config in cfg.PLATFORM_CONFIGS.items():

    if not os.path.exists(config["OutputPath"]):
        os.makedirs(config["OutputPath"])

    ppq_acc,ort_acc,platform_acc = None,None,None

    print(f'---------------------- PPQ Quantization Test Running with {model_name} on {platform}----------------------')
    model_path = f'{os.path.join(cfg.FP32_BASE_PATH, model_name)}-FP32.onnx'
    
    # 量化onnx模型
    ppq_quant_ir = quantize_onnx_model(
        onnx_import_file=model_path, calib_dataloader=dataloader, calib_steps=cfg.CALIBRATION_NUM // cfg.BATCHSIZE, 
        setting=config["QuanSetting"],input_shape=cfg.INPUT_SHAPE, collate_fn=lambda x: x.to(cfg.DEVICE), 
        platform=config["QuantPlatform"], do_quantize=True)
        
    # 评估PPQ模拟量化准确度
    if cfg.GET_PPQ_ACC:
        ppq_acc,_ = get_ppq_accuracy(model_name,platform,ppq_quant_ir)

    # 评估ORT模型准确度
    if cfg.GET_ORT_ACC:
        # 导出ORT模型
        export_ppq_graph(
            graph = ppq_quant_ir,
            platform=TargetPlatform.ORT_OOS_INT8,
            graph_save_to=f'{os.path.join(config["OutputPath"], model_name)}-ORT-INT8.onnx')

        ort_acc,_ = get_ort_accuracy(model_name,platform,config["OutputPath"])


    # 评估目标平台部署准确度
    if cfg.GET_PLATFORM_ACC:
        # 导出平台模型,暂时无法和ORT一起导出
        export_ppq_graph(
            graph = ppq_quant_ir,
            platform=config["ExportPlatform"],
            graph_save_to=f'{os.path.join(config["OutputPath"], model_name)}-{platform}-INT8.onnx')

        if platform in {"OpenVino","TRT"}:
            platform_acc,_  = get_platform_accuracy(model_name,platform,config["OutputPath"])
        else:
            platform_acc = None


    report.append([model_name,platform,fp32_acc,ppq_acc,ort_acc,platform_acc])

report = pd.DataFrame(report,columns=["model","paltform","fp32_acc","ppq_acc","ort_acc","platform_acc"])
print("-------------------测试报告如下---------------------")
print(report)
report.to_csv(f"{cfg.BASE_PATH}/report.csv",index=False)

In [2]:
import os
import torch
import torchvision.transforms as transforms
import torchvision

In [3]:
data_path= "/home/geng/fiftyone/coco-2017/validation/data"
labels_path="/home/geng/fiftyone/coco-2017/validation/labels.json"
transform = transforms.Compose([
    transforms.Resize(size=(320,320)),
    transforms.ToTensor(),
])
# def collate_fn_coco(batch):
#     return tuple(zip(*batch))
# coco 数据集存在空标签
coco_dataset = torchvision.datasets.CocoDetection(data_path, labels_path, transform=transform)
coco_dataloader = torch.utils.data.DataLoader(coco_dataset,batch_size=1,num_workers=0)

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [3]:
coco_dataset[250][1]

[]

In [6]:
from torchvision.io.image import read_image
from torchvision.models.detection import ssd300_vgg16,SSD300_VGG16_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

img = read_image("color.jpg")

# Step 1: Initialize model with the best available weights
weights = SSD300_VGG16_Weights.COCO_V1
model = ssd300_vgg16(weights=weights,score_thresh=0.95)
model.eval()

# Step 2: Initialize the inference transforms

# Step 3: Apply inference preprocessing transforms
# Step 4: Use the model and visualize the prediction


SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [7]:
from tqdm import tqdm
res = []
for images,labels in tqdm(coco_dataloader):
    if not labels: continue
    output = model(images)[0]
    if len(labels) == 0:
        print("error!")
        print(images,labels)
    for box in output["boxes"]:
        res.append({
            'score': output["scores"],
            'category_id': output["labels"],
            'bbox': box,
            'image_id': labels[0]["image_id"]
        })

 64%|██████▍   | 3213/5000 [10:52<51:21,  1.72s/it]  

: 

: 

In [35]:
import json
b = json.dump({'annotations':res})
with open("prediction.json","w") as f:
    f.write(b)

[{'score': tensor([0.9830], grad_fn=<IndexBackward0>),
  'category_id': tensor([72]),
  'bbox': tensor([[  3.6491, 115.9564,  71.7471, 186.6564]], grad_fn=<StackBackward0>),
  'image_id': tensor([139])}]

In [41]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
from tempfile import NamedTemporaryFile

# json file in coco format, original annotation data
anno_file =labels_path
coco_gt = COCO(anno_file)

# Use GT box as prediction box for calculation, the purpose is to get detection_res
with open(anno_file,'r') as f:
    json_file = json.load(f)
annotations = json_file['annotations']

# detection_res = []
detection_res = res

for anno in annotations:
    detection_res.append({
        'score': 1.,
        'category_id': anno['category_id'],
        'bbox': anno['bbox'],
        'image_id': anno['image_id']
    })
with NamedTemporaryFile(suffix='.json') as tf:
    # Due to subsequent needs, first convert detection_res to binary and then write it to the json file
    content = json.dumps(detection_res).encode(encoding='utf-8')
    tf.write(content)
    res_path = tf.name

    # loadRes will generate a new COCO type instance based on coco_gt and return
    coco_dt = coco_gt.loadRes(res_path)

    cocoEval = COCOeval(coco_gt, coco_dt,'bbox')
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()

print(cocoEval.stats)

loading annotations into memory...
Done (t=0.95s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.14s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=25.17s).
Accumulating evaluation results...
DONE (t=3.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.563
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.976
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet